In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### import libs


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import tensorflow.keras as k

import csv
import cv2
import os

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.densenet import DenseNet201 ,DenseNet121
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.applications import VGG16,VGG19
from tensorflow.keras.applications.densenet import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow import keras
from tensorflow.keras import layers


from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder , OneHotEncoder
from sklearn.model_selection import train_test_split

from scipy import ndimage

In [2]:
imagetype= os.listdir("../input/emotion-detection-fer/train")
test_imagetype= os.listdir("../input/emotion-detection-fer/test")
ck_imagetype= os.listdir("../input/ckplus/CK+48")
imagetype.remove("disgusted")
test_imagetype.remove("disgusted")
# ck_imagetype.remove("contempt")
print(imagetype)
print(test_imagetype)
print(ck_imagetype)

In [3]:
images = []    #images for train list will store all images
labels= []
t_images = []    #images for test list will store all images
t_labels= []
ck_images = []
ck_labels = []
path = "../input/emotion-detection-fer/train/"
t_path = "../input/emotion-detection-fer/test/"
ck_path = "../input/ckplus/CK+48"

## **FER13**

### *Load data*

In [4]:
for i in imagetype:
    print(i)
    flower_path=path+str(i)
    data_list=[j for j in os.listdir(flower_path) if j.endswith(".png") ]
    counter=0
    for data in data_list:
#         if counter> 2390: break
        img=cv2.imread(flower_path + '/'+ data)
#         img = cv2.resize(img , (64,64))
        images.append(img)
        labels.append(i)
    
print("total train images:",len(images))
print("total train images:",len(labels))


In [5]:
for i in test_imagetype:
    print(i)
    flower_path=t_path+str(i)
    test_data_list=[j for j in os.listdir(flower_path) if j.endswith(".png") ]
    counter=0
    for data in test_data_list:
#         if counter> 2390: break
        img=cv2.imread(flower_path + '/'+ data)
#         img = cv2.resize(img , (64,64))
        t_images.append(img)
        t_labels.append(i)
print("total test images:",len(t_images))
print("total test images:",len(t_labels))

In [6]:
# images,labels = shuffle(images, labels, random_state=10)
# t_image , t_labels =  shuffle(t_images, t_labels, random_state=10)


# for i in range(0,2000):
#     images.append(t_images[i])
#     labels.append(t_labels[i])

In [7]:
type(images)

In [8]:
encode= LabelEncoder()
labels=encode.fit_transform(labels)
encode2= LabelEncoder()
t_labels = encode2.fit_transform(t_labels)
t_labels[1:10]

In [9]:
labels=labels.reshape(-1,1)
onehotencoder= OneHotEncoder()
labels=onehotencoder.fit_transform(labels)
labels=labels.toarray()
labels
print("train labels is ready!")

In [10]:
t_labels=t_labels.reshape(-1,1)
onehotencoder= OneHotEncoder()
t_labels=onehotencoder.fit_transform(t_labels)
t_labels=t_labels.toarray()
print("test labels is ready!")
t_labels[:10]

In [11]:
images=np.array(images)
print(images.shape)
print(labels.shape)

In [12]:
t_images=np.array(t_images)
print(t_images.shape)
print(t_labels.shape)

In [13]:
images,labels = shuffle(images, labels, random_state=10)
t_images , t_labels =  shuffle(t_images, t_labels, random_state=10)

In [14]:
plt.imshow(images[4])
plt.xlabel(labels[4])

## **CKPLUS**

### *load data*

In [ ]:
for i in ck_imagetype:
    print(i)
    flower_path=ck_path+"/"+str(i)
    data_list=[j for j in os.listdir(flower_path) if j.endswith(".png") ]
    counter=0
    for data in data_list:
#         if counter> 2390: break
        img=cv2.imread(flower_path + '/'+ data)
#         img = cv2.resize(img , (64,64))
        images.append(img)
        labels.append(i)
    
print("total train images:",len(images))
print("total train images:",len(labels))

In [ ]:
print("start encoding", end=" ---> ")
encode= LabelEncoder()
labels=encode.fit_transform(labels)
print ("encoding finished")
print ("start one hot encoding", end=" ---> ")
labels=labels.reshape(-1,1)
onehotencoder= OneHotEncoder()
labels=onehotencoder.fit_transform(labels)
labels=labels.toarray()
print("one hot encoding finished")
images=np.array(images)
print("images and list shapes:")
print("images shape:", end=" " )
print(images.shape)
print("labels shape:", end=" " )
print(labels.shape)


In [ ]:
images,labels = shuffle(images, labels, random_state=10)
plt.imshow(images[4])
plt.xlabel(labels[4])

### *DenseNet201*

In [ ]:
DenseNet201 = DenseNet201(input_shape= (48,48,3) ,  include_top=False , weights="imagenet", pooling='max')

In [ ]:
DenseNet201.trainable = False
x = Flatten()(DenseNet201.output)
prediction = Dense(7, activation='softmax')(x)

# create a model object
DenseNet201 = Model(inputs=DenseNet201.input, outputs=prediction)

In [ ]:
opt = keras.optimizers.Adam(learning_rate=0.00010000000474974513)
DenseNet201.compile(
  loss='categorical_crossentropy',
  optimizer=opt,
  metrics=['accuracy']
)

In [ ]:
r = DenseNet201.fit(images[736:] , labels[736:] , batch_size=32 , epochs=50 , validation_data = (images[:245] , labels[:245]))

In [ ]:
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
# plt.savefig('LossVal_loss_DenseNet201')

# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()

### *MobileNet*

In [ ]:
mobile = MobileNet(input_shape= (48,48,3) , dropout = 0.1 ,  include_top=False , weights="imagenet", pooling='max')

In [ ]:
mobile.trainable = False
x = Flatten()(mobile.output)
prediction = Dense(7, activation='softmax')(x)
mobile = Model(inputs=mobile.input, outputs=prediction)

In [ ]:
opt = keras.optimizers.Adam(learning_rate=0.00010000000474974513)
mobile.compile(
  loss='categorical_crossentropy',
  optimizer=opt,
  metrics=['accuracy']
)

In [ ]:
r = mobile.fit(images[736:] , labels[736:] , batch_size=32 , epochs=50 , validation_data = (images[:245] , labels[:245]))

In [ ]:
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
# plt.savefig('LossVal_loss_DenseNet201')

# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()

### *VGG16*

In [ ]:
vgg16 = VGG16(input_shape= (48,48,3) , include_top=False , weights="imagenet", pooling='max')


In [ ]:
vgg16.trainable = True
x = Flatten()(vgg16.output)
prediction = Dense(7, activation='softmax')(x)

# create a model object
vgg16 = Model(inputs=vgg16.input, outputs=prediction)

In [ ]:
opt = keras.optimizers.Adam(learning_rate=0.00010000000474974513)
vgg16.compile(
  loss='categorical_crossentropy',
  optimizer=opt,
  metrics=['accuracy']
)

In [ ]:
r = vgg16.fit(images[736:] , labels[736:] , batch_size=32 , epochs=50 , validation_data = (images[:245] , labels[:245]))

In [ ]:
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
# plt.savefig('LossVal_loss_DenseNet201')

# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()

In [ ]:
# vgg16.save_weights('vgg16_ckplus.h5')
# 

## **models**

*NO.1*

In [ ]:
model = keras.Sequential([
# Data Augmentation 
  

  DenseNet121(weights='imagenet',include_top=False, classes=6, input_shape=(48,48,3)), #Extract features 

  layers.BatchNormalization(),


#CNN layer 

  layers.Conv2D(128,(3,3),padding = 'same'), 
  layers.BatchNormalization(),
  layers.LeakyReLU(alpha=0.1),
  layers.MaxPooling2D(pool_size = (2,2),padding = 'same'), 
  layers.Dropout(0.65), 


  layers.Flatten(),
#Fully connected 1st layer 

  layers.Dense(256),
  layers.BatchNormalization(), 
  layers.Activation('relu'), 
  layers.Dropout(0.65), 

# Fully connected layer 2nd layer

#   layers.Dense(128),
#   layers.BatchNormalization(), 
#   layers.Activation('relu'), 
#   layers.Dropout(0.5), 


  layers.Dense(7, activation='softmax'),
                          
])
model.summary()

In [ ]:
opt = keras.optimizers.Adam(learning_rate=0.00010000000474974513)
model.compile(
  loss='categorical_crossentropy',
  optimizer=opt,
  metrics=['accuracy']
)


In [ ]:
history = model.fit(images[736:] , labels[736:] , batch_size=32 , epochs=60 , validation_data = (images[:245] , labels[:245]))

In [ ]:
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.legend()
plt.show()
# plt.savefig('LossVal_loss_DenseNet201')

# plot the accuracy
plt.plot(history.history['accuracy'], label='train acc')
plt.plot(history.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()

*NO.2*

In [ ]:
from tensorflow.keras.layers import Activation, Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization
model2 = tf.keras.Sequential()

#Block 1:
model2.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal',input_shape=(48,48,3)))
model2.add(Activation('relu'))
model2.add(BatchNormalization())
# model2.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
# model2.add(Activation('relu'))
# model2.add(BatchNormalization())
model2.add(MaxPooling2D(pool_size=(2,2)))
model2.add(Dropout(0.65))

#Block 2:
model2.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'))
model2.add(Activation('relu'))
model2.add(BatchNormalization())
# model2.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'))
# model2.add(Activation('relu'))
# model2.add(BatchNormalization())
model2.add(MaxPooling2D(pool_size=(2,2)))
model2.add(Dropout(0.65))

# #block 3:
model2.add(Conv2D(512,(3,3),padding='same',kernel_initializer='he_normal'))
model2.add(Activation('relu'))
model2.add(BatchNormalization())
model2.add(MaxPooling2D(pool_size=(2,2)))
model2.add(Dropout(0.65))


model2.add(Flatten())
model2.add(Dense(1024,kernel_initializer='he_normal'))
model2.add(Activation('relu'))
model2.add(BatchNormalization())
model.add(Dropout(0.65))


model2.add(Dense(1024,kernel_initializer='he_normal'))
model2.add(Activation('relu'))
model2.add(BatchNormalization())
model2.add(Dropout(0.65))


model2.add(Dense(7,kernel_initializer='he_normal'))
model2.add(Activation('softmax'))

model2.summary()

In [ ]:
opt = keras.optimizers.Adam(learning_rate=0.00010000000474974513)
model2.compile(
  loss='categorical_crossentropy',
  optimizer=opt,
  metrics=['accuracy']
)


In [ ]:
history = model2.fit(images[736:] , labels[736:] , batch_size=32 , epochs=50 , validation_data = (images[:245] , labels[:245]))

In [ ]:
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.legend()
plt.show()
# plt.savefig('LossVal_loss_DenseNet201')

# plot the accuracy
plt.plot(history.history['accuracy'], label='train acc')
plt.plot(history.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()

## Data Augmentation


In [ ]:
datagen =  ImageDataGenerator(
  #preprocessing_function=preprocess_input,
  rotation_range=30,
  width_shift_range=0.2,
  height_shift_range=0.2,
  shear_range=0.2,
  zoom_range=0.2,
  horizontal_flip=True
)

In [ ]:
aug = ImageDataGenerator(rotation_range=45, width_shift_range=0.2,
                        height_shift_range=0.2, horizontal_flip = False,
                        fill_mode='nearrest')

In [ ]:
aug.fit(images)


In [ ]:
history = model.fit(images , labels, batch_size=32 , epochs=60 , validation_data = (t_images , t_labels))

### VGG16



In [ ]:
model = keras.Sequential([
# Data Augmentation 
  

  VGG16(weights='imagenet',include_top=False, classes=6, input_shape=(64,64,3)), #Extract features 

  layers.BatchNormalization(),


#CNN layer 

  layers.Conv2D(64,(3,3),padding = 'same'), 
  layers.BatchNormalization(),
  layers.LeakyReLU(alpha=0.1),
  layers.MaxPooling2D(pool_size = (2,2),padding = 'same'), 
  layers.Dropout(0.65), 
    
#    layers.Conv2D(128,(3,3),padding = 'same'), 
#   layers.BatchNormalization(),
#   layers.LeakyReLU(alpha=0.1),
#   layers.MaxPooling2D(pool_size = (2,2),padding = 'same'), 
#   layers.Dropout(0.65),
    
#  layers.Conv2D(512,(3,3),padding = 'same'), 
#  layers.BatchNormalization(),
#  layers.LeakyReLU(alpha=0.1),
#  layers.MaxPooling2D(pool_size = (2,2),padding = 'same'), 
#  layers.Dropout(0.65), 
    
 layers.Flatten(),
#Fully connected 1st layer 

  layers.Dense(1000),
  layers.BatchNormalization(), 
  layers.Activation('relu'), 
  layers.Dropout(0.65), 

# Fully connected layer 2nd layer

#   layers.Dense(128),
#   layers.BatchNormalization(), 
#   layers.Activation('relu'), 
#   layers.Dropout(0.5), 


  layers.Dense(6, activation='softmax'),
                          
])
model.summary()

In [ ]:
# model.trainable = False
# dropout1 = Dropout(0.5)
# x = Flatten()(model.output)
# x= x = dropout1(model.output)
# prediction = Dense(6, activation='softmax')(x)
# model = Model(inputs=model.input, outputs=prediction)

In [ ]:
# opt = keras.optimizers.Adam(learning_rate=0.00010000000474974513)
opt = keras.optimizers.Adam(learning_rate=1e-7)
model.compile(
  loss='categorical_crossentropy',
  optimizer=opt,
  metrics=['accuracy']
)


In [ ]:
history = model.fit(images , labels, batch_size=128 , epochs=30 , validation_data = (t_images , t_labels))

In [ ]:
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.legend()
plt.show()
# plt.savefig('LossVal_loss_DenseNet201')

# plot the accuracy
plt.plot(history.history['accuracy'], label='train acc')
plt.plot(history.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()

### NASNetLarge

In [ ]:
nas = NASNetLarge(
    input_shape=(64,64,3),
    include_top=True,
#     weights="imagenet",
    pooling='max',
#     classes=6
)

In [ ]:
nas.trainable = False
x = Flatten()(nas.output)
prediction = Dense(7, activation='softmax')(x)

# create a model object
nas = Model(inputs=nas.input, outputs=prediction)

In [ ]:
opt = keras.optimizers.Adam(learning_rate=0.00010000000474974513)
nas.compile(
  loss='categorical_crossentropy',
  optimizer=opt,
  metrics=['accuracy']
)
history = nas.fit(images , labels, batch_size=32 , epochs=30 , validation_data = (t_images , t_labels))

#### efficientNet B0

In [17]:
ef_b0 = tf.keras.applications.EfficientNetB0(input_shape= (48,48,3) , include_top=False , weights="imagenet", pooling='max')

In [21]:
ef_b0.trainable = False
x = Flatten()(ef_b0.output)
prediction = Dense(6, activation='softmax')(x)

# create a model object
ef_b0 = Model(inputs=ef_b0.input, outputs=prediction)

# learning rate 
opt = keras.optimizers.Adam(learning_rate=0.00010000000474974513)
ef_b0.compile(
  loss='categorical_crossentropy',
  optimizer=opt,
  metrics=['accuracy']
)

In [22]:
opt = keras.optimizers.Adam(learning_rate=0.00010000000474974513)
ef_b0.compile(
  loss='categorical_crossentropy',
  optimizer=opt,
  metrics=['accuracy']
)

In [23]:
r = ef_b0.fit(images[736:] , labels[736:] , batch_size=32 , epochs=50 , validation_data = (images[:245] , labels[:245]))

In [24]:
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
# plt.savefig('LossVal_loss_DenseNet201')

# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()

In [25]:
ef_b1 = tf.keras.applications.EfficientNetB1(input_shape= (48,48,3) , include_top=False , weights="imagenet", pooling='max')

################## B1

ef_b1.trainable = False
x = Flatten()(ef_b1.output)
prediction = Dense(6, activation='softmax')(x)

# create a model object
ef_b1 = Model(inputs=ef_b1.input, outputs=prediction)

# learning rate 
opt = keras.optimizers.Adam(learning_rate=0.00010000000474974513)
ef_b1.compile(
  loss='categorical_crossentropy',
  optimizer=opt,
  metrics=['accuracy']
)

In [27]:
r = ef_b1.fit(images[736:] , labels[736:] , batch_size=32 , epochs=50 , validation_data = (images[:245] , labels[:245]))

In [28]:
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
# plt.savefig('LossVal_loss_DenseNet201')

# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()

#### EfficentNet B2

In [32]:
ef_b2 = tf.keras.applications.EfficientNetB2(input_shape= (48,48,3) , include_top=False , weights="imagenet", pooling='max')

################## B1

ef_b2.trainable = False
x = Flatten()(ef_b2.output)
prediction = Dense(6, activation='softmax')(x)

# create a model object
ef_b2 = Model(inputs=ef_b2.input, outputs=prediction)

# learning rate 
opt = keras.optimizers.Adam(learning_rate=0.00010000000474974513)
ef_b2.compile(
  loss='categorical_crossentropy',
  optimizer=opt,
  metrics=['accuracy']
)

In [33]:
r = ef_b2.fit(images , labels , batch_size=32 , epochs=50 , validation_data = (t_images , t_labels))

In [35]:
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
# plt.savefig('LossVal_loss_DenseNet201')

# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()

In [36]:
ef_b3 = tf.keras.applications.EfficientNetB3(input_shape= (48,48,3) , include_top=False , weights="imagenet", pooling='max')

################## B1

ef_b3.trainable = False
x = Flatten()(ef_b3.output)
prediction = Dense(6, activation='softmax')(x)

# create a model object
ef_b3 = Model(inputs=ef_b3.input, outputs=prediction)

# learning rate 
opt = keras.optimizers.Adam(learning_rate=0.00010000000474974513)
ef_b3.compile(
  loss='categorical_crossentropy',
  optimizer=opt,
  metrics=['accuracy']
)

In [37]:
r = ef_b3.fit(images , labels , batch_size=32 , epochs=50 , validation_data = (t_images , t_labels))

In [38]:
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
# plt.savefig('LossVal_loss_DenseNet201')

# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()

In [39]:
ef_b4 = tf.keras.applications.EfficientNetB4(input_shape= (48,48,3) , include_top=False , weights="imagenet", pooling='max')

################## B1

ef_b4.trainable = False
x = Flatten()(ef_b4.output)
prediction = Dense(6, activation='softmax')(x)

# create a model object
ef_b4 = Model(inputs=ef_b4.input, outputs=prediction)

# learning rate 
opt = keras.optimizers.Adam(learning_rate=0.00010000000474974513)
ef_b4.compile(
  loss='categorical_crossentropy',
  optimizer=opt,
  metrics=['accuracy']
)

In [40]:
r = ef_b3.fit(images , labels , batch_size=32 , epochs=50 , validation_data = (t_images , t_labels))

In [42]:
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
# plt.savefig('LossVal_loss_DenseNet201')

# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()